# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [51]:
#Your code here
import pandas as pd

df = pd.read_csv('homepage_actions.csv')

clicked = df.loc[df.action == 'click', 'id']
click_count = clicked.count()

viewed = df.loc[df.action =='view', 'id']
view_count = viewed.count()

clicked_not_viewed = set(clicked) - set(viewed)
#no one clicked but didn't view

overlap = set(df.loc[df.group == 'experiment', 'id']).intersection(set(df.loc[df.group == 'control', 'id']))
#there's no overlap between control and experiment

set()

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [167]:
#Your code here
import scipy.stats as stats

experiment = df[df.group == 'experiment']
control = df[df.group == 'control']


experiment['action_num'] = experiment['action'].replace({'view': 0, 'click': 1})
control['action_num'] = control['action'].replace({'view': 0, 'click': 1})

obs_clicked = experiment.groupby('id').sum()['action_num'].value_counts()
exp_clicked = control.groupby('id').sum()['action_num'].value_counts()

stats.chisquare(obs_clicked, exp_clicked)


/var/folders/ws/w4d9z41n7vj1pg3kqv95frm80000gn/T/ipykernel_49905/3627074236.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experiment['action_num'] = experiment['action'].replace({'view': 0, 'click': 1})
/var/folders/ws/w4d9z41n7vj1pg3kqv95frm80000gn/T/ipykernel_49905/3627074236.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['action_num'] = control['action'].replace({'view': 0, 'click': 1})


ValueError: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1e-08, but the percent differences are:
0.11214953271028037

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [196]:
#Your code here
control_rate = exp_clicked.loc[1] / exp_clicked.loc[0]
experiment_rate = obs_clicked.loc[1] / obs_clicked.loc[0]
print('if experiment had same click-through rate as control: ', obs_clicked.loc[0] * control_rate)
print(f'control_rate:{control_rate}, experiment_rate:{experiment_rate}')

if experiment had same click-through rate as control:  803.0733333333333
control_rate:0.3883333333333333, experiment_rate:0.44874274661508706


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [201]:
#Your code here
control_std = control.action_num.std()
experiment_std = experiment.action_num.std()

control_mean = control.action_num.mean()
z_score = experiment_rate / control_std
z_score

1.0856835400675275

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [210]:
#Your code here
stats.norm.cdf(z_score, loc=control_mean, scale=control_std)

0.9820419298679817

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.